In [19]:
import sys
from pathlib import Path


# Add the parent directory to the Python path
sys.path.append(str(Path().resolve().parent))

from spytial import *
from spytial.annotations import *


# Binary Search Trees

In [20]:
@attribute(field="key")
@orientation(selector='{ x, y : BSTNode | (y.key not in NoneType) and x.left = y }',
             directions=['below','left'])
@orientation(selector='{ x, y : BSTNode | (y.key not in NoneType) and x.right = y }',
             directions=['below','right'])
class BSTNode:
    def __init__(self, key=None, left=None, right=None, parent=None):
        self.key = key
        self.left = left
        self.right = right
        self.parent = parent

# Singleton NIL sentinel
BST_NIL = BSTNode(key=None)
BST_NIL.left = BST_NIL.right = BST_NIL.parent = BST_NIL

@flag(name='hideDisconnected')
@hideAtom(selector='{ x : BSTNode | (x.key in NoneType) }')   # hide NIL
@hideAtom(selector='BSTree')                                  # hide wrapper
@hideField(field='parent')                                    # hide pointers
class BSTree:
    def __init__(self):
        self.root = BST_NIL

    # TREE-SEARCH
    def search(self, k):
        x = self.root
        while x is not BST_NIL and k != x.key:
            x = x.left if k < x.key else x.right
        return x

    # TREE-MINIMUM / MAXIMUM
    def minimum(self, x):
        while x.left is not BST_NIL:
            x = x.left
        return x
    def maximum(self, x):
        while x.right is not BST_NIL:
            x = x.right
        return x

    # TREE-SUCCESSOR
    def successor(self, x):
        if x.right is not BST_NIL:
            return self.minimum(x.right)
        y = x.parent
        while y is not BST_NIL and x is y.right:
            x, y = y, y.parent
        return y

    # TREE-INSERT
    def insert(self, k):
        z = BSTNode(key=k, left=BST_NIL, right=BST_NIL, parent=None)
        y, x = BST_NIL, self.root
        while x is not BST_NIL:
            y = x
            x = x.left if z.key < x.key else x.right
        z.parent = y
        if y is BST_NIL: self.root = z
        elif z.key < y.key: y.left = z
        else: y.right = z
        return z

    # helper: TRANSPLANT
    def _transplant(self, u, v):
        if u.parent is BST_NIL: self.root = v
        elif u is u.parent.left: u.parent.left = v
        else: u.parent.right = v
        v.parent = u.parent

    # TREE-DELETE
    def delete(self, z):
        if z.left is BST_NIL:
            self._transplant(z, z.right)
        elif z.right is BST_NIL:
            self._transplant(z, z.left)
        else:
            y = self.minimum(z.right)     # successor
            if y.parent is not z:
                self._transplant(y, y.right)
                y.right = z.right
                y.right.parent = y
            self._transplant(z, y)
            y.left = z.left
            y.left.parent = y

    # inorder (for sanity checks)
    def inorder(self, node=None, acc=None):
        if node is None:
            node, acc = self.root, []
        if node is BST_NIL:
            return acc
        self.inorder(node.left, acc)
        acc.append(node.key)
        self.inorder(node.right, acc)
        return acc


In [21]:

t = BSTree()

# insert keys (any iterable of comparable keys works)
for k in [10, 20, 30, 15, 25, 5, -10, 0]:
    print(f"Inserting {k}...")
    t.insert(k)
    diagram(t) # Shows you each step of the insertion




Inserting 10...


Inserting 20...


Inserting 30...


Inserting 15...


Inserting 25...


Inserting 5...


Inserting -10...


Inserting 0...


# Red Black Trees

In [22]:

RED = "red"
BLACK = "black"

@attribute(field="key")
@attribute(field="color")
@orientation(selector='{ x, y : RBNode| (y.key not in NoneType) and x.left = y}', directions=['below', 'left'])
@orientation(selector='{ x, y : RBNode |  (y.key not in NoneType) and x.right = y}', directions=['below', 'right'])
@atomColor(
    selector='{ x : RBNode | @:(x.color) = red }',
    value='red'
)
@atomColor(
    selector='{ x : RBNode | @:(x.color) = black }',
    value='black'
)
class RBNode:
    def __init__(self, key=None, color=BLACK, left=None, right=None, parent=None):
        self.key = key
        self.color = color
        self.left = left
        self.right = right
        self.parent = parent

# Singleton NIL sentinel (all leaves point here)
NIL = RBNode(key=None, color=BLACK)
NIL.left = NIL.right = NIL.parent = NIL

@flag(name='hideDisconnected')
@hideAtom(selector='{ x : RBNode | (x.key in NoneType) }') # Hide the NIL node
@hideAtom(selector='RBTree') # Hide the pointer / root thing
@hideField(field='parent') # Hide parent pointers
class RBTree:
    def __init__(self):
        self.root = NIL

    # ----- Utility -----
    def search(self, key):
        x = self.root
        while x is not NIL and key != x.key:
            x = x.left if key < x.key else x.right
        return x

    def minimum(self, x):
        while x.left is not NIL:
            x = x.left
        return x

    # ----- Rotations (CLRS §13.2) -----
    def left_rotate(self, x):
        y = x.right
        assert y is not NIL, "left_rotate requires x.right != NIL"
        x.right = y.left
        if y.left is not NIL:
            y.left.parent = x
        y.parent = x.parent
        if x.parent is NIL:
            self.root = y
        elif x is x.parent.left:
            x.parent.left = y
        else:
            x.parent.right = y
        y.left = x
        x.parent = y

    def right_rotate(self, y):
        x = y.left
        assert x is not NIL, "right_rotate requires y.left != NIL"
        y.left = x.right
        if x.right is not NIL:
            x.right.parent = y
        x.parent = y.parent
        if y.parent is NIL:
            self.root = x
        elif y is y.parent.left:
            y.parent.left = x
        else:
            y.parent.right = x
        x.right = y
        y.parent = x

    # ----- Insert + Fixup (CLRS §13.3) -----
    def insert(self, key):
        z = RBNode(key=key, color=RED, left=NIL, right=NIL, parent=None)
        y = NIL
        x = self.root
        # BST insert
        while x is not NIL:
            y = x
            x = x.left if z.key < x.key else x.right
        z.parent = y
        if y is NIL:
            self.root = z
        elif z.key < y.key:
            y.left = z
        else:
            y.right = z
        # Fix red-black properties
        self._insert_fixup(z)
        return z

    def _insert_fixup(self, z):
        while z.parent.color is RED:
            if z.parent is z.parent.parent.left:
                y = z.parent.parent.right  # uncle
                if y.color is RED:
                    # Case 1
                    z.parent.color = BLACK
                    y.color = BLACK
                    z.parent.parent.color = RED
                    z = z.parent.parent
                else:
                    if z is z.parent.right:
                        # Case 2
                        z = z.parent
                        self.left_rotate(z)
                    # Case 3
                    z.parent.color = BLACK
                    z.parent.parent.color = RED
                    self.right_rotate(z.parent.parent)
            else:
                # mirror image
                y = z.parent.parent.left
                if y.color is RED:
                    z.parent.color = BLACK
                    y.color = BLACK
                    z.parent.parent.color = RED
                    z = z.parent.parent
                else:
                    if z is z.parent.left:
                        z = z.parent
                        self.right_rotate(z)
                    z.parent.color = BLACK
                    z.parent.parent.color = RED
                    self.left_rotate(z.parent.parent)
        self.root.color = BLACK

    # ----- (Optional) Traversal for testing -----
    def inorder(self, node=None, acc=None):
        if node is None:
            node, acc = self.root, []
        if node is NIL:
            return acc
        self.inorder(node.left, acc)
        acc.append((node.key, node.name))
        self.inorder(node.right, acc)
        return acc


In [23]:

t = RBTree()

# insert keys (any iterable of comparable keys works)
for k in [10, 20, 30, 15, 25, 5, -10, 0]:
    print(f"Inserting {k}...")
    t.insert(k)
    diagram(t) # Shows you each step of the insertion




Inserting 10...


Inserting 20...


Inserting 30...


Inserting 15...


Inserting 25...


Inserting 5...


Inserting -10...


Inserting 0...


# B Trees

TODO: Better selectors

In [24]:


#@attribute(field="keys")
@attribute(field="leaf")
@orientation(selector='{ x, y : BNode | (! @bool:(x.leaf)) and (y in x.children) }',
             directions=['below'])   # children render below their parent (left→right by index)
class BNode:
    def __init__(self, leaf=True):
        self.keys: list[int] = []        # sorted
        self.children: list["BNode"] = []# len = len(keys)+1 if not leaf
        self.leaf: bool = leaf

@flag(name='hideDisconnected')
@hideAtom(selector='BTree')   # hide wrapper
class BTree:
    def __init__(self, t: int):
        assert t >= 2, "t >= 2"
        self.t = t
        self.root = BNode(leaf=True)

    # B-TREE-SEARCH
    def search(self, k, x=None):
        if x is None: x = self.root
        i = 0
        while i < len(x.keys) and k > x.keys[i]:
            i += 1
        if i < len(x.keys) and k == x.keys[i]:
            return (x, i)
        return None if x.leaf else self.search(k, x.children[i])

    # split child x.children[i] (full) into y (left) and z (right), lift median
    def _split_child(self, x: BNode, i: int):
        t = self.t
        y = x.children[i]
        z = BNode(leaf=y.leaf)
        mid = y.keys[t-1]
        z.keys = y.keys[t:]         # right t-1 keys
        y.keys = y.keys[:t-1]       # left t-1 keys
        if not y.leaf:
            z.children = y.children[t:]
            y.children = y.children[:t]
        x.children.insert(i+1, z)
        x.keys.insert(i, mid)

    # insert into nonfull node
    def _insert_nonfull(self, x: BNode, k: int):
        i = len(x.keys) - 1
        if x.leaf:
            x.keys.append(None)
            while i >= 0 and k < x.keys[i]:
                x.keys[i+1] = x.keys[i]
                i -= 1
            x.keys[i+1] = k
            return
        while i >= 0 and k < x.keys[i]:
            i -= 1
        i += 1
        if len(x.children[i].keys) == 2*self.t - 1:
            self._split_child(x, i)
            if k > x.keys[i]: i += 1
        self._insert_nonfull(x.children[i], k)

    # B-TREE-INSERT
    def insert(self, k: int):
        r = self.root
        if len(r.keys) == 2*self.t - 1:
            s = BNode(leaf=False)
            s.children = [r]
            self.root = s
            self._split_child(s, 0)
            self._insert_nonfull(s, k)
        else:
            self._insert_nonfull(r, k)


In [25]:
t = BTree(3)

# insert keys (any iterable of comparable keys works)
for k in [10, 20, 30, 15, 25, 5, -10, 0]:
    print(f"Inserting {k}...")
    t.insert(k)
    diagram(t) # Shows you each step of the insertion




Inserting 10...


Inserting 20...


Inserting 30...


Inserting 15...


Inserting 25...


Inserting 5...


Inserting -10...


Inserting 0...


# Fibonacci Heaps (priority queues)

## TODO: Is BUGGY I THINK

In [26]:
import math

## BUGGY -- this is wrong I think (?)

# Fibonacci Heap (priority queue) with spatial annotations for visualization
@attribute(field="key")
@attribute(field="degree")
@attribute(field="mark")
@orientation(selector='{ x, y : FibNode | (y.key not in NoneType) and y.parent = x }',              directions=['below'])
@orientation(selector='{ x, y : FibNode | (x.parent in FibNode) and (y.key not in NoneType) and x.right = y }',             directions=['right'])
@orientation(selector='{ x, y : FibNode | (x.parent in FibNode) and (y.key not in NoneType) and x.left = y }',              directions=['left'])
@atomColor(selector='{ x : FibNode | @bool:(x.mark) }', value='red')
class FibNode:
    def __init__(self, key=None):
        self.key = key
        self.degree = 0
        self.mark = False
        self.parent = None
        self.child = None
        # circular doubly-linked list pointers
        self.left = self
        self.right = self

@flag(name='hideDisconnected')
@attribute(field="n")
@hideAtom(selector='NoneType')
# @hideAtom(selector='FibonacciHeap')   # hide wrapper
# @hideField(field='parent')            # hide parent pointers in visualization
class FibonacciHeap:
    def __init__(self):
        self.min: FibNode | None = None
        self.n: int = 0

    # helper: iterate a circular doubly-linked list starting at node
    def _iterate(self, start: FibNode):
        if start is None:
            return
        node = start
        while True:
            yield node
            node = node.right
            if node is start:
                break

    # insert a new key, return the node
    def insert(self, key):
        x = FibNode(key=key)
        # add to root list
        if self.min is None:
            self.min = x
        else:
            # insert x to the right of min
            x.right = self.min.right
            x.left = self.min
            self.min.right.left = x
            self.min.right = x
            if x.key < self.min.key:
                self.min = x
        self.n += 1
        return x

    def find_min(self):
        return self.min

    # merge another heap into this one (destructive)
    def union(self, other: "FibonacciHeap"):
        if other is None or other.min is None:
            return
        if self.min is None:
            self.min = other.min
            self.n = other.n
            return
        # concatenate root lists
        a = self.min.right
        b = other.min.left
        self.min.right = other.min
        other.min.left = self.min
        a.left = b
        b.right = a
        if other.min.key < self.min.key:
            self.min = other.min
        self.n += other.n

    # extract the minimum node and return it
    def extract_min(self):
        z = self.min
        if z is not None:
            # move z's children to root list
            if z.child is not None:
                children = list(self._iterate(z.child))
                for x in children:
                    # remove parent link
                    x.parent = None
                    # splice x into root list (to the right of min)
                    x.left = self.min
                    x.right = self.min.right
                    self.min.right.left = x
                    self.min.right = x
            # remove z from root list
            if z is z.right:
                self.min = None
            else:
                z.left.right = z.right
                z.right.left = z.left
                self.min = z.right
                self._consolidate()
            self.n -= 1
        return z

    def _consolidate(self):
        if self.min is None:
            return
        # upper bound on degree
        max_degree = int(math.log(self.n, 2)) + 2 if self.n > 0 else 1
        A = [None] * (max_degree + 1)
        roots = list(self._iterate(self.min))
        for w in roots:
            x = w
            d = x.degree
            while A[d] is not None:
                y = A[d]
                if x.key > y.key:
                    x, y = y, x
                self._link(y, x)
                A[d] = None
                d += 1
            A[d] = x
        # rebuild root list and find new min
        self.min = None
        for node in A:
            if node is not None:
                # isolate node
                node.left = node.right = node
                if self.min is None:
                    self.min = node
                else:
                    # insert to root list
                    node.right = self.min.right
                    node.left = self.min
                    self.min.right.left = node
                    self.min.right = node
                    if node.key < self.min.key:
                        self.min = node

    def _link(self, y: FibNode, x: FibNode):
        # remove y from root list
        y.left.right = y.right
        y.right.left = y.left
        # make y a child of x
        y.parent = x
        y.left = y.right = y
        if x.child is None:
            x.child = y
        else:
            # insert y into x's child circular list
            y.right = x.child.right
            y.left = x.child
            x.child.right.left = y
            x.child.right = y
        x.degree += 1
        y.mark = False

    def decrease_key(self, x: FibNode, k):
        if k > x.key:
            raise ValueError("new key is greater than current key")
        x.key = k
        y = x.parent
        if y is not None and x.key < y.key:
            self._cut(x, y)
            self._cascading_cut(y)
        if x.key < self.min.key:
            self.min = x

    def _cut(self, x: FibNode, y: FibNode):
        # remove x from y's child list
        if y.child is x:
            # if x is the only child
            if x.right is x:
                y.child = None
            else:
                y.child = x.right
        x.left.right = x.right
        x.right.left = x.left
        y.degree -= 1
        # add x to root list
        x.left = self.min
        x.right = self.min.right
        self.min.right.left = x
        self.min.right = x
        x.parent = None
        x.mark = False

    def _cascading_cut(self, y: FibNode):
        z = y.parent
        if z is not None:
            if not y.mark:
                y.mark = True
            else:
                self._cut(y, z)
                self._cascading_cut(z)

    def delete(self, x: FibNode):
        # decrease key to -infinity then extract min
        self.decrease_key(x, -float("inf"))
        self.extract_min()

# Demo: insert keys and visualize after each step
t = FibonacciHeap()
for k in [10, 20, 30, 15, 25, 5, -10, 0]:
    print(f"Inserting {k}...")
    t.insert(k)
    diagram(t)  # visualizes each insertion step

Inserting 10...


Inserting 20...


Inserting 30...


Inserting 15...


Inserting 25...


Inserting 5...


Inserting -10...


Inserting 0...


# van Emde Boas (vEB) Tree (CLRS Ch. 20)

TODO